In [1]:
# basic
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import time
import glob
import os
from numpy.linalg import inv
from bidict import bidict

# user-defined
import sys
sys.path.append('/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/codes/')
from module import *
from module.backtesting import *
from module.agent_network import *
from module.environment import market_envrionment
from module.data_handler import *
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# visulization
import igraph
import cairocffi
# import cairo
import matplotlib.pyplot as plt

# 
from scipy.sparse.csgraph import minimum_spanning_tree

In [2]:
bt = vectorized_backtesting()

In [3]:
bt.tbn_combined

12141   12138   12635   12850    3144    3413   14225    3897  \
1996 12141  1.0000  0.0323  0.0266  0.0818  0.0000  0.0000  0.0105  0.0000   
     12138  0.0323  1.0000  0.0349  0.0000  0.0038  0.0000  0.0000  0.0000   
     12635  0.0266  0.0349  1.0000  0.0590  0.0000  0.0000  0.0878  0.0000   
     12850  0.0818  0.0000  0.0590  1.0000  0.0000  0.0000  0.0260  0.0000   
     3144   0.0000  0.0038  0.0000  0.0000  1.0000  0.0041  0.0000  0.0000   
...            ...     ...     ...     ...     ...     ...     ...     ...   
1998 16478  0.0000  0.0156  0.0000  0.0000  0.0065  0.0092  0.0000  0.0415   
     11687  0.0195  0.0309  0.0776  0.0000  0.0000  0.0109  0.0094  0.0000   
     2184   0.0592  0.0329  0.0020  0.0000  0.0000  0.0038  0.1289  0.0000   
     4494   0.0148  0.0000  0.0492  0.0000  0.0660  0.0000  0.0284  0.0086   
     11811  0.0510  0.0073  0.0321  0.0221  0.0000  0.0032  0.0000  0.0000   

            4199   17120  ...    4213   12726   13510   15267   15414   16478  \
1996 12141   0.0  0.0076  ...  0.0000  0.0000  0.0000  0.0058  0.0036  0.0000   
     12138   0.0  0.0584  ...  0.0000  0.0581  0.0219  0.0000  0.0000  0.0000   
     12635   0.0  0.0374  ...  0.0000  0.0178  0.0254  0.0010  0.0000  0.0000   
     12850   0.0  0.0000  ...  0.0000  0.0000  0.0000  0.0000  0.0308  0.0000   
     3144    0.0  0.0070  ...  0.0485  0.0000  0.0000  0.0000  0.0000  0.0207   
...          ...     ...  ...     ...     ...     ...     ...     ...     ...   
1998 16478   0.0  0.0325  ...  0.0010  0.0328  0.0000  0.0331  0.0000  1.0000   
     11687   0.0  0.1784  ...  0.0000  0.0409  0.0000  0.0000  0.0000  0.0205   
     2184    0.0  0.0111  ...  0.0000  0.0037  0.0495  0.0247  0.0000  0.0000   
     4494    0.0  0.0000  ...  0.0000  0.0000  0.0092  0.0312  0.0000  0.0063   
     11811   0.0  0.0553  ...  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   

             11687    2184    4494   11811  
1996 12141  0.0000  0.0620  0.0000  0.0000  
     12138  0.0835  0.0172  0.0084  0.0160  
     12635  0.0467  0.0683  0.0391  0.0348  
     12850  0.0000  0.0148  0.0018  0.0348  
     3144   0.0001  0.0000  0.0138  0.0000  
...            ...     ...     ...     ...  
1998 16478  0.0205  0.0000  0.0063  0.0000  
     11687  1.0000  0.0000  0.0000  0.0402  
     2184   0.0000  1.0000  0.0000  0.0084  
     4494   0.0000  0.0000  1.0000  0.0000  
     11811  0.0402  0.0084  0.0000  1.0000  

[2952 rows x 123 columns]

In [3]:
sample_size = 100
company_subset_gvkey_vector_100 = np.random.choice(db.company_gvkey_vector, sample_size, replace=False)
db.extract_TNIC_network_subset(key_list= company_subset_gvkey_vector_100,
                                file_input_path='/Users/cheng/Documents/Research Data/Text Base Network/new/tnic_all_data/',
                                file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_100/')

100%|██████████| 32/32 [16:31<00:00, 30.98s/it]


In [4]:
sample_size = 50
company_subset_gvkey_vector_50 = np.random.choice(db.company_gvkey_vector, sample_size, replace=False)
db.extract_TNIC_network_subset(key_list= company_subset_gvkey_vector_50,
                                file_input_path='/Users/cheng/Documents/Research Data/Text Base Network/new/tnic_all_data/',
                                file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_50/')

100%|██████████| 32/32 [09:58<00:00, 18.70s/it]
